In [11]:
import pymysql
import pandas as pd

# MySQL 연결
conn = pymysql.connect(
    host='triodb.crgkkewom2kz.ap-northeast-2.rds.amazonaws.com',
    user='lee',
    password='trio1234',
    db='triodb',
    charset='utf8mb4'
)

# 구별 점포 수 CSV
query_gu = """SELECT ca.gu AS 구,
       sca.name AS 업종명,
       SUM(s.shop_count) AS 점포수
FROM Shop_Count s
JOIN Commercial_Area ca ON s.commercial_area_code = ca.code
JOIN Service_Category sca ON s.service_category_code = sca.code
WHERE sca.name IN ('한식음식점','중식음식점','일식음식점','양식음식점',
                   '제과점','패스트푸드점','치킨전문점','분식전문점',
                   '호프-간이주점','커피-음료')
GROUP BY ca.gu, sca.name
ORDER BY ca.gu, 점포수 DESC;"""
df_gu = pd.read_sql(query_gu, conn)
df_gu.to_csv('./src/eda/shop_count_by_gu.csv', index=False, encoding='utf-8-sig')

# 동별 점포 수 CSV
query_dong = """SELECT ca.gu AS 구,
       ca.dong AS 동,
       sca.name AS 업종명,
       SUM(s.shop_count) AS 점포수
FROM Shop_Count s
JOIN Commercial_Area ca ON s.commercial_area_code = ca.code
JOIN Service_Category sca ON s.service_category_code = sca.code
WHERE sca.name IN ('한식음식점','중식음식점','일식음식점','양식음식점',
                   '제과점','패스트푸드점','치킨전문점','분식전문점',
                   '호프-간이주점','커피-음료')
GROUP BY ca.gu, ca.dong, sca.name
ORDER BY ca.gu, ca.dong, 점포수 DESC;"""
df_dong = pd.read_sql(query_dong, conn)
df_dong.to_csv('./src/eda/shop_count_by_dong.csv', index=False, encoding='utf-8-sig')

# 상권별 점포 수 CSV
query_area = """SELECT ca.gu AS 구,
       ca.name AS 상권명,
       sca.name AS 업종명,
       SUM(s.shop_count) AS 점포수
FROM Shop_Count s
JOIN Commercial_Area ca ON s.commercial_area_code = ca.code
JOIN Service_Category sca ON s.service_category_code = sca.code
WHERE sca.name IN ('한식음식점','중식음식점','일식음식점','양식음식점',
                   '제과점','패스트푸드점','치킨전문점','분식전문점',
                   '호프-간이주점','커피-음료')
GROUP BY ca.gu, ca.name, sca.name
ORDER BY ca.gu, 점포수 DESC;"""
df_area = pd.read_sql(query_area, conn)
df_area.to_csv('./src/eda/shop_count_by_area.csv', index=False, encoding='utf-8-sig')

conn.close()

/tmp/ipykernel_8355/1264956762.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_gu = pd.read_sql(query_gu, conn)
/tmp/ipykernel_8355/1264956762.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dong = pd.read_sql(query_dong, conn)
/tmp/ipykernel_8355/1264956762.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_area = pd.read_sql(query_area, conn)


## 구/동/상권별 CSV 파일 불러오기

In [12]:
import pandas as pd

# 구/동/상권별 CSV
df_gu = pd.read_csv('./src/eda/shop_count_by_gu.csv', encoding='utf-8-sig')
df_dong = pd.read_csv('./src/eda/shop_count_by_dong.csv', encoding='utf-8-sig')
df_area = pd.read_csv('./src/eda/shop_count_by_area.csv', encoding='utf-8-sig')

#### 구별 업종 점포수 시각화

In [13]:
import plotly.express as px

fig_gu = px.bar(df_gu,
                x='구',
                y='점포수',
                color='업종명',
                text='점포수',
                title='구별 10개 업종 점포 수',
                barmode='group')
fig_gu.update_layout(xaxis_title='구', yaxis_title='점포 수', legend_title='업종')
fig_gu.show()

#### 동별 업종 점포수 시각화

In [14]:
fig_dong = px.treemap(df_dong,
                      path=['구', '동', '업종명'],
                      values='점포수',
                      color='점포수',
                      color_continuous_scale='Blues',
                      title='동별 10개 업종 점포 수')
fig_dong.show()

#### 상권별 업종 점포수 시각화

점포수 상위 20개 상권

In [20]:
top_n = df_area.groupby('상권명')['점포수'].sum().sort_values(ascending=False).head(20).index
df_top = df_area[df_area['상권명'].isin(top_n)]

fig_top = px.bar(df_top,
                 x='상권명',
                 y='점포수',
                 color='업종명',
                 title='점포 수 상위 20개 상권 (업종별)',
                 text='점포수')
fig_top.update_layout(xaxis_title='상권', yaxis_title='점포 수', legend_title='업종', height=600)
fig_top.show()